### Maxwell-Boltzmann Distribution

In [ ]:
import numpy as np
import scipy as sy
from numba import jit # optional to speed up code; comment out if not installed

Define a function to create a set a randomly distributed particles.

In [ ]:
def initial_conditions(N, v0):
    
    # create arrays of random numbers for (x, y) locations of each particle
    x, y = np.random.random(N), np.random.random(N)
    # create arrays of zeros for the initial velocities of each particle
    vx, vy = np.zeros_like(x), np.zeros_like(y)
    # set one particle with high initial velocity
    idx = x**2 + y**2 == np.min(x**2 + y**2)
    vx[idx], vy[idx] = v0, v0
    
    return x, y, vx, vy

Define a function to apply classical dynamics for a single time step $dt$.

In [ ]:
@jit(nopython=True) # comment out this line if numba not installed
def classical_step(x, y, vx, vy, dt, N):
    
    # create arrays for updated locations and velocities
    x_, y_, vx_, vy_ = np.zeros_like(x), np.zeros_like(y), np.zeros_like(vx), np.zeros_like(vy)
    
    # consider forces acting on each particle
    for i in range(len(x)):
        # set acceleration on the i'th particle to zero (initially)
        ax, ay = 0., 0.
        
        # estimate the electrostactic force on the i'th particle from each other particle
        for j in range(len(x)):
            # sum contribution from this particle to x- and y-components of the acceleration
            if not j == i:
                ax = ax + 1/(2*np.pi*np.sqrt(N))*(x[i] - x[j])/((x[i] - x[j])**2 + (y[i] - y[j])**2)
                ay = ay + 1/(2*np.pi*np.sqrt(N))*(y[i] - y[j])/((x[i] - x[j])**2 + (y[i] - y[j])**2)
                    # SI units are multiplied by factor of sqrt(N)*q**2/epsilon_0
                    # note this is Coulomb's Law for a 2D plane
            
        # calculate updated location and velocity of i'th particle
        x_[i] = x[i] + vx[i]*dt + 1/2.*ax*dt**2
        y_[i] = y[i] + vy[i]*dt + 1/2.*ay*dt**2
        vx_[i] = vx[i] + ax*dt
        vy_[i] = vy[i] + ay*dt
        
        # reflect particle off the boundary if it leaves the box during the time step
        # will not work if particles travel across width of box in less than a single time step
        if x_[i] < 0:
            x_[i] = -x_[i]
            vx_[i] = -vx_[i]
        if x_[i] > 1:
            x_[i] = 2 - x_[i]
            vx_[i] = -vx_[i]
        if y_[i] < 0:
            y_[i] = -y_[i]
            vy_[i] = -vy_[i]
        if y_[i] > 1:
            y_[i] = 2 - y_[i]
            vy_[i] = -vy_[i]
        
    return x_, y_, vx_, vy_

Define function to apply numerous iterations of the classical dynamics function.

In [ ]:
def simualtion(N, t, dt, v0):
    
    # initial conditions for particles
    x, y, vx, vy = initial_conditions(N, v0)
    # create lists to store time, location and velocity outputs for each time step 
    t_array, x_array, y_array, vx_array, vy_array = [0.], [x], [y], [vx], [vy]
    
    # evaluate classical dynamics for total time
    tc, i = 0., 0
    while tc < t:
        # update current time step, ensuring final step is exactly t
        if tc + dt > t:
            dt = t - tc
            tc = t
        else:
            tc = tc + dt
        i = i + 1
        
        # run classical dynamics for a time step
        x, y, vx, vy = classical_step(x, y, vx, vy, dt, N)
        # add new time step to output arrays
        t_array.append(tc)
        x_array.append(x)
        y_array.append(y)
        vx_array.append(vx)
        vy_array.append(vy)
        
    return t_array, x_array, y_array, vx_array, vy_array

Run the simulation. Ensure $dt$ is sufficiently small that particles have not escaped the box by the final time step.

In [ ]:
t_array, x_array, y_array, vx_array, vy_array = simualtion(100, 10, 0.00001, 1e5)

In [ ]:
print(t_array[-1], x_array[-1], y_array[-1], vx_array[-1], vy_array[-1])

Define a function to plot the velocity distribution.

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import rc

def velocity_plotter(vx, vy):

    # set up latex labels on plot (optional)
    try:
        rc('text', usetex=True) # can try usetex=False
        rc('font', size=14)
        rc('legend', fontsize=14)
        rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
    except:
        pass

    # create figure
    fig, ax = plt.subplots(figsize=(6, 6))

    # set axis labels
    ax.set_xlabel(r'Velocity (arbitrary units)')
    ax.set_ylabel(r'Number of particles')

    ax.hist(np.sqrt(np.asarray(vx).flatten()**2 + np.asarray(vy).flatten()**2), bins=np.arange(0, 2, 0.025), color='red')

    plt.show()

In [ ]:
velocity_plotter(vx_array[len(vx_array)//2:-1], vy_array[len(vy_array)//2:-1])